<a href="https://colab.research.google.com/github/muraleee/collab-stuff/blob/main/whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls

In [ ]:
!sudo apt update && sudo apt install ffmpeg

In [ ]:
!wget -O SL-e01-Part1.mp3 https://raw.githubusercontent.com/muraleee/saundlaha/refs/heads/main/SL-e01-Part1.mp3


In [1]:
!pip install -q git+https://github.com/openai/whisper.git


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# import whisper
# import os
# import time

# def transcribe_with_stats(file_path, model_type="large"):
#     if not os.path.exists(file_path):
#         print(f"Error: {file_path} not found.")
#         return

#     # Load model
#     print(f"Loading Whisper {model_type} model...")
#     model = whisper.load_model(model_type)

#     # Start Timer
#     start_time = time.perf_counter()

#     print(f"Transcribing: {file_path}...")
#     try:
#         result = model.transcribe(file_path)

#         # End Timer
#         end_time = time.perf_counter()
#         total_seconds = end_time - start_time

#         # Save to file
#         base_name = os.path.splitext(file_path)[0]
#         output_file = f"{base_name}.txt"
#         with open(output_file, "w", encoding="utf-8") as f:
#             f.write(result["text"].strip())

#         # Format and display time
#         mins, secs = divmod(total_seconds, 60)
#         print("-" * 30)
#         print(f"✅ Success!")
#         print(f"📄 Saved to: {output_file}")
#         print(f"⏱️ Time taken: {int(mins)}m {secs:.2f}s")
#         print("-" * 30)

#     except Exception as e:
#         print(f"An error occurred: {e}")

# if __name__ == "__main__":
#     audio_file = "./saundlaha/SL-e01-Part5.mp3"
#     transcribe_with_stats(audio_file)

In [2]:
import whisper
import os
import time
import json

def transcribe_saundarya_lahari(file_path):
    if not os.path.exists(file_path):
        print(f"Error: {file_path} not found.")
        return

    # Use 'large' or 'turbo' for best Sanskrit phonetics
    model_type = "large"
    print(f"Loading {model_type} model...")
    model = whisper.load_model(model_type)

    # Specialized Prompt for Saundarya Lahari
    # Includes specific mantras and names to prime the AI
    sl_prompt = (
        "This is a discourse on the Saundarya Lahari Text authored by Adi Shankaracharya. "
        "It includes Sanskrit verses, Shlokas, and terms like Devi, Devata, Chandrasekhar, Tanras, Vedas and many other Indian spiritual terms from Sanskrit"
        "The speaker has an Indian accent. Additionally,"
        "there are parts of sanskrit verse fragments in the audio they maybe interspersed between English text"

    )

    print(f"Transcribing: {file_path}...")
    start_time = time.perf_counter()

    try:
        result = model.transcribe(
                audio=file_path,
                language="en",                        # Stick to English commentary
                initial_prompt=sl_prompt,
                word_timestamps=True,                 # Pinpoint the verses exactly
                beam_size=5,                          # Higher accuracy for technical terms
                condition_on_previous_text=False      # Prevents "looping" on complex Sanskrit
            )
        elapsed = time.perf_counter() - start_time

        # Save output
        output_file = os.path.splitext(file_path)[0] + ".txt"
        with open(output_file, "w", encoding="utf-8") as f:
            f.write(result["text"].strip())

        output_json = os.path.splitext(file_path)[0] + ".json"

        with open(output_json, "w", encoding="utf-8") as f:
            # indent=4 makes the file human-readable
            # ensure_ascii=False keeps Devanagari/Sanskrit characters intact
            json.dump(result, f, indent=4, ensure_ascii=False)

        print(f"✅ Completed in {elapsed/60:.1f} minutes.")
        print(f"📄 Text saved to: {output_file}")
        print(f"📄 Text in JSON saved to: {output_json}")

    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    audio_file = "./saundlaha/SL-e01-Part5.mp3"
    transcribe_saundarya_lahari(audio_file)

Loading large model...


100%|██████████████████████████████████████| 2.88G/2.88G [00:14<00:00, 220MiB/s]


Transcribing: ./saundlaha/SL-e01-Part5.mp3...
✅ Completed in 3.5 minutes.
📄 Text saved to: ./saundlaha/SL-e01-Part5.txt
📄 Text in JSON saved to: ./saundlaha/SL-e01-Part5.json


In [ ]:
# import torch
# import os
# from transformers import pipeline

# # 1. Environment & Device Setup
# # H100/A100/L4 benefit from float16 precision
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
# torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# model_id = "ARTPARK-IISc/whisper-large-v3-vaani-hindi"

# print(f"Loading model on {device}...")
# transcribe = pipeline(
#     task="automatic-speech-recognition",
#     model=model_id,
#     device=device,
#     torch_dtype=torch_dtype,
#     model_kwargs={"attn_implementation": "sdpa"} # Faster attention for A100/H100/L4
# )

# # 2. Prepare specialized prompt
# audio_file = "./saundlaha/SL-e01-Part2.mp3"
# sl_prompt = (
#     "This is a discourse on the Saundarya Lahari by Adi Shankaracharya. शिवः शक्त्या युक्तो "
#     "It includes Sanskrit verses, Shlokas, and terms like Devi, Devata, Chandrasekhar, "
#     "Chakras, Tantras, and Vedas. The speaker has an Indian accent."
# )

# # Fix: Convert prompt IDs to PyTorch Tensor to avoid NumPy TypeError
# prompt_ids = transcribe.tokenizer.get_prompt_ids(sl_prompt, return_tensors="pt")
# prompt_ids = prompt_ids.to(device)

# print(f"Transcribing: {audio_file}...")
# result = transcribe(
#     audio_file,
#     chunk_length_s=30,
#     generate_kwargs={"prompt_ids": prompt_ids}
# )

# # 3. Save to file
# output_dir = "./saundlaha/"
# os.makedirs(output_dir, exist_ok=True)
# output_file = os.path.join(output_dir, "Vaani_Part2.txt")

# with open(output_file, "w", encoding="utf-8") as f:
#     f.write(result["text"])

# print(f"✅ Success! Transcription saved to {output_file}")